# ReEDS Scenarios on PV ICE Tool

To explore different scenarios for furture installation projections of PV (or any technology), ReEDS output data can be useful in providing standard scenarios. ReEDS installation projections are used in this journal as input data to the PV ICE tool. 

Current sections include:

<ol>
    <li> ### Reading a standard ReEDS output file </li>
    <li> ### Saving PCA data as PV ICE input format </li>
    <li> ### Saving State data as PV ICE input format </li>
</ol>


In [1]:
import numpy as np
import pandas as pd
import os,sys
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 22})
plt.rcParams['figure.figsize'] = (12, 8)

In [2]:
import os
from pathlib import Path

testfolder = str(Path().resolve().parent.parent.parent / 'PV_ICE' / 'TEMP')

print ("Your simulation will be stored in %s" % testfolder)


Your simulation will be stored in C:\Users\sayala\Documents\GitHub\CircularEconomy-MassFlowCalculator\PV_ICE\TEMP


## Reading a standard ReEDS output file

In [3]:
reedsFile = str(Path().resolve().parent.parent.parent.parent / 'December Core Scenarios ReEDS Outputs Solar Futures v2a.xlsx')
print ("Input file is stored in %s" % reedsFile)

Input file is stored in C:\Users\sayala\Documents\GitHub\December Core Scenarios ReEDS Outputs Solar Futures v2a.xlsx


In [4]:
REEDSInput = pd.read_excel(reedsFile,
                        sheet_name="UPV Capacity (GW)")
                        #index_col=[0,2,3]) #this casts scenario, PCA and State as levels


## Save Input Files by PCA

#### Create a copy of the REEDS Input and modify structure for PCA focus

In [5]:
rawdf = REEDSInput.copy()
rawdf.drop(columns=['State'], inplace=True)
rawdf.drop(columns=['Tech'], inplace=True)
rawdf.set_index(['Scenario','Year','PCA'], inplace=True)
rawdf.head(21)

Capacity (GW)
Scenario      Year PCA               
Reference.Mod 2010 p1        0.000000
              2012 p1        0.000000
              2014 p1        0.000000
              2016 p1        0.000000
              2018 p1        0.000000
              2020 p1        0.025000
              2022 p1        0.025000
              2024 p1        0.025000
              2026 p1        0.025000
              2028 p1        0.025000
              2030 p1        0.025000
              2032 p1        0.025000
              2034 p1        0.025000
              2036 p1        0.025000
              2038 p1        0.025000
              2040 p1        0.025000
              2042 p1        0.025000
              2044 p1        0.025000
              2046 p1        0.025000
              2048 p1        0.025000
              2050 p1        0.282881

#### Loading Module Baseline. Will be used later to populate all the columsn otehr than 'new_Installed_Capacity_[MW]' which will be supplied by the REEDS model

In [6]:
import PV_ICE
r1 = PV_ICE.Simulation(name='Simulation1', path=testfolder)
r1.createScenario(name='US', file=r'..\baselines\ReedsSubset\baseline_modules_US_Reeds.csv')
baseline = r1.scenario['US'].data
baseline = baseline.drop(columns=['new_Installed_Capacity_[MW]'])
baseline.set_index('year', inplace=True)
baseline.index = pd.PeriodIndex(baseline.index, freq='A')  # A -- Annual
baseline.head()

path = C:\Users\sayala\Documents\GitHub\CircularEconomy-MassFlowCalculator\PV_ICE\TEMP


,mod_eff,mod_reliability_t50,mod_reliability_t90,mod_degradation,mod_lifetime,mod_MFG_eff,mod_EOL_collection_eff,mod_EOL_collected_recycled,mod_Repowering,mod_Repairing
year,,,,,,,,,,
2009,14.577778,24.0,29.0,0.6,21.6,98.0,0.0,0.0,0.0,0.0
2010,14.700000,25.0,30.0,0.3,22.7,98.0,0.0,0.0,0.0,0.0
2011,15.100000,25.0,30.0,0.3,23.0,98.0,0.0,0.0,0.0,0.0
2012,15.400000,25.0,30.0,0.3,23.5,98.0,0.0,0.0,0.0,0.0
2013,16.000000,28.0,33.0,0.3,24.2,98.0,0.0,0.0,0.0,0.0


#### For each Scenario and for each PCA, combine with baseline and save as input file

In [7]:
for ii in range (len(rawdf.unstack(level=1))):
    PCA = rawdf.unstack(level=1).iloc[ii].name[1]
    SCEN = rawdf.unstack(level=1).iloc[ii].name[0]
    SCEN=SCEN.replace('+', '_')
    filetitle = SCEN+'_'+PCA +'.csv'
    filetitle = os.path.join(testfolder, 'PCAs', filetitle)
    A = rawdf.unstack(level=1).iloc[ii]
    A = A.droplevel(level=0)
    A.name = 'new_Installed_Capacity_[MW]'
    A = pd.DataFrame(A)
    A.index=pd.PeriodIndex(A.index, freq='A')
    A = A.resample('Y').asfreq()
    A = A['new_Installed_Capacity_[MW]'].fillna(0).groupby(A['new_Installed_Capacity_[MW]'].notna().cumsum()).transform('mean')    
    A = pd.DataFrame(A)
    A['new_Installed_Capacity_[MW]'] = A['new_Installed_Capacity_[MW]'] * 1000   # ReEDS file is in GW.
    # Add other columns
    A = pd.concat([A, baseline.reindex(A.index)], axis=1)
    A.loc['2050']['new_Installed_Capacity_[MW]'] = A.loc['2050']['new_Installed_Capacity_[MW]']/2   # Dividing last value by 2
    new_row = A[-1:]
    new_row.index = new_row.index.shift(periods=1)   #Shifting so new row is 2051
    A = pd.concat([A,pd.concat([new_row])])              
    A.index = A.index.shift(periods=-1)          # Shifting back so it goes from 2009-2050

    
    header = "year,new_Installed_Capacity_[MW],mod_eff,mod_reliability_t50,mod_reliability_t90,"\
    "mod_degradation,mod_lifetime,mod_MFG_eff,mod_EOL_collection_eff,mod_EOL_collected_recycled,"\
    "mod_Repowering,mod_Repairing\n"\
    "year,MW,%,years,years,%,years,%,%,%,%,%\n"

    with open(filetitle, 'w', newline='') as ict:
    # Write the header lines, including the index variable for
    # the last one if you're letting Pandas produce that for you.
    # (see above).
        for line in header:
            ict.write(line)

        #    savedata.to_csv(ict, index=False)
        A.to_csv(ict, header=False)

## Save Input Files By States

#### Reassign data from REEDS Input, as we need one of the columns we dropped.

In [8]:
rawdf = REEDSInput.copy()
#rawdf.drop(columns=['State'], inplace=True)
rawdf.drop(columns=['Tech'], inplace=True)
rawdf.set_index(['Scenario','Year','PCA', 'State'], inplace=True)
rawdf.head(21)


Capacity (GW)
Scenario      Year PCA State               
Reference.Mod 2010 p1  WA          0.000000
              2012 p1  WA          0.000000
              2014 p1  WA          0.000000
              2016 p1  WA          0.000000
              2018 p1  WA          0.000000
              2020 p1  WA          0.025000
              2022 p1  WA          0.025000
              2024 p1  WA          0.025000
              2026 p1  WA          0.025000
              2028 p1  WA          0.025000
              2030 p1  WA          0.025000
              2032 p1  WA          0.025000
              2034 p1  WA          0.025000
              2036 p1  WA          0.025000
              2038 p1  WA          0.025000
              2040 p1  WA          0.025000
              2042 p1  WA          0.025000
              2044 p1  WA          0.025000
              2046 p1  WA          0.025000
              2048 p1  WA          0.025000
              2050 p1  WA          0.282881

#### Group data so we can work with the States instead

In [9]:
df = rawdf.groupby(['Scenario','State', 'Year'])['Capacity (GW)'].sum(axis=0)
df = pd.DataFrame(df)
df.head()

Capacity (GW)
Scenario          State Year               
95-by-35+Elec.Adv AL    2010         0.0000
                        2012         0.0000
                        2014         0.0000
                        2016         0.0750
                        2018         0.1941

#### For each Scenario and for each STATE, combine with baseline and save as input file

In [10]:
for ii in range (len(df.unstack(level=2))):
    STATE = df.unstack(level=2).iloc[ii].name[1]
    SCEN = df.unstack(level=2).iloc[ii].name[0]
    SCEN=SCEN.replace('+', '_')
    filetitle = SCEN+'_'+STATE +'.csv'
    filetitle = os.path.join(testfolder, 'STATEs', filetitle)
    A = df.unstack(level=2).iloc[ii]
    A = A.droplevel(level=0)
    A.name = 'new_Installed_Capacity_[MW]'
    A = pd.DataFrame(A)
    A.index=pd.PeriodIndex(A.index, freq='A')
    A = A.resample('Y').asfreq()
    A = A['new_Installed_Capacity_[MW]'].fillna(0).groupby(A['new_Installed_Capacity_[MW]'].notna().cumsum()).transform('mean')    
    A = pd.DataFrame(A)
    A['new_Installed_Capacity_[MW]'] = A['new_Installed_Capacity_[MW]'] * 1000   # ReEDS file is in GW.
    # Add other columns
    A = pd.concat([A, baseline.reindex(A.index)], axis=1)
    A.loc['2050']['new_Installed_Capacity_[MW]'] = A.loc['2050']['new_Installed_Capacity_[MW]']/2   # Dividing last value by 2
    new_row = A[-1:]
    new_row.index = new_row.index.shift(periods=1)   #Shifting so new row is 2051
    A = pd.concat([A,pd.concat([new_row])])              
    A.index = A.index.shift(periods=-1)          # Shifting back so it goes from 2009-2050

    
    header = "year,new_Installed_Capacity_[MW],mod_eff,mod_reliability_t50,mod_reliability_t90,"\
    "mod_degradation,mod_lifetime,mod_MFG_eff,mod_EOL_collection_eff,mod_EOL_collected_recycled,"\
    "mod_Repowering,mod_Repairing\n"\
    "year,MW,%,years,years,%,years,%,%,%,%,%\n"

    with open(filetitle, 'w', newline='') as ict:
    # Write the header lines, including the index variable for
    # the last one if you're letting Pandas produce that for you.
    # (see above).
        for line in header:
            ict.write(line)

        #    savedata.to_csv(ict, index=False)
        A.to_csv(ict, header=False)

## Method with PIVOT tables for STATES (REDUNDANT)

In [11]:
"""
df = pd.read_excel(reedsFile,
                        sheet_name="UPV Capacity (GW)")
                        #index_col=[0,2,3]) #this casts scenario, PCA and State as levels
df.drop(columns=['Tech'], inplace=True)

#foo = df.pivot_table(index=['Scenario','Year'],columns='State',aggfunc=sum)
foo = df.pivot_table(index=['Scenario','State'],columns='Year',aggfunc=sum)

foo = pd.DataFrame(foo)
foo.head()


for ii in range (len(foo)):
    STATE = foo.index[ii][1]
    SCEN = foo.index[ii][0]
    SCEN=SCEN.replace('+', '_')
    filetitle = SCEN+'_'+STATE +'.csv'
    filetitle = os.path.join(testfolder, 'STATEs', filetitle)
    A = pd.DataFrame(foo.iloc[ii])
    #A = A.unstack(level=0).iloc[ii]
    A = A.droplevel(level=0)
    A.columns = [' '.join(col).strip() for col in A.columns.values]
    A.rename(columns = {list(A)[0]: 'new_Installed_Capacity_[MW]'}, inplace = True)
    A.index=pd.PeriodIndex(A.index, freq='A')
    A = A.resample('Y').asfreq()
    A = A['new_Installed_Capacity_[MW]'].fillna(0).groupby(A['new_Installed_Capacity_[MW]'].notna().cumsum()).transform('mean')    
    A = pd.DataFrame(A)
    A['new_Installed_Capacity_[MW]'] = A['new_Installed_Capacity_[MW]'] * 1000   # ReEDS file is in GW.
    # Add other columns
    A = pd.concat([A, baseline.reindex(A.index)], axis=1)
    A.loc['2050']['new_Installed_Capacity_[MW]'] = A.loc['2050']['new_Installed_Capacity_[MW]']/2   # Dividing last value by 2
    new_row = A[-1:]
    new_row.index = new_row.index.shift(periods=1)   #Shifting so new row is 2051
    A = pd.concat([A,pd.concat([new_row])])              
    A.index = A.index.shift(periods=-1)          # Shifting back so it goes from 2009-2050

    header = "year,new_Installed_Capacity_[MW],mod_eff,mod_reliability_t50,mod_reliability_t90,"\
    "mod_degradation,mod_lifetime,mod_MFG_eff,mod_EOL_collection_eff,mod_EOL_collected_recycled,"\
    "mod_Repowering,mod_Repairing\n"\
    "year,MW,%,years,years,%,years,%,%,%,%,%\n"

    with open(filetitle, 'w', newline='') as ict:
    # Write the header lines, including the index variable for
    # the last one if you're letting Pandas produce that for you.
    # (see above).
        for line in header:
            ict.write(line)

        #    savedata.to_csv(ict, index=False)
        A.to_csv(ict, header=False)
        
"""
pass

# Saving US Baseline

### Create a copy of the REEDS Input and modify structure for PCA focus

In [12]:
rawdf = REEDSInput.copy()
#rawdf.drop(columns=['State'], inplace=True)
rawdf.drop(columns=['Tech'], inplace=True)
rawdf.set_index(['Scenario','Year'], inplace=True)
rawdf.head(21)

PCA State  Capacity (GW)
Scenario      Year                         
Reference.Mod 2010  p1    WA       0.000000
              2012  p1    WA       0.000000
              2014  p1    WA       0.000000
              2016  p1    WA       0.000000
              2018  p1    WA       0.000000
              2020  p1    WA       0.025000
              2022  p1    WA       0.025000
              2024  p1    WA       0.025000
              2026  p1    WA       0.025000
              2028  p1    WA       0.025000
              2030  p1    WA       0.025000
              2032  p1    WA       0.025000
              2034  p1    WA       0.025000
              2036  p1    WA       0.025000
              2038  p1    WA       0.025000
              2040  p1    WA       0.025000
              2042  p1    WA       0.025000
              2044  p1    WA       0.025000
              2046  p1    WA       0.025000
              2048  p1    WA       0.025000
              2050  p1    WA       0.282881

In [13]:
df = rawdf.groupby(['Scenario','Year'])['Capacity (GW)'].sum(axis=0)


### Loading Module Baseline. Will be used later to populate all the columsn other than 'new_Installed_Capacity_[MW]' which will be supplied by the REEDS model

In [14]:
import PV_ICE
r1 = PV_ICE.Simulation(name='Simulation1', path=testfolder)
r1.createScenario(name='US', file=r'..\baselines\ReedsSubset\baseline_modules_US_Reeds.csv')
baseline = r1.scenario['US'].data
baseline = baseline.drop(columns=['new_Installed_Capacity_[MW]'])
baseline.set_index('year', inplace=True)
baseline.index = pd.PeriodIndex(baseline.index, freq='A')  # A -- Annual
baseline.head()

path = C:\Users\sayala\Documents\GitHub\CircularEconomy-MassFlowCalculator\PV_ICE\TEMP


,mod_eff,mod_reliability_t50,mod_reliability_t90,mod_degradation,mod_lifetime,mod_MFG_eff,mod_EOL_collection_eff,mod_EOL_collected_recycled,mod_Repowering,mod_Repairing
year,,,,,,,,,,
2009,14.577778,24.0,29.0,0.6,21.6,98.0,0.0,0.0,0.0,0.0
2010,14.700000,25.0,30.0,0.3,22.7,98.0,0.0,0.0,0.0,0.0
2011,15.100000,25.0,30.0,0.3,23.0,98.0,0.0,0.0,0.0,0.0
2012,15.400000,25.0,30.0,0.3,23.5,98.0,0.0,0.0,0.0,0.0
2013,16.000000,28.0,33.0,0.3,24.2,98.0,0.0,0.0,0.0,0.0


### For each Scenario, combine with baseline and save as input file¶

In [15]:
for ii in range (len(df.unstack(level=1))):
    SCEN = df.unstack(level=1).index[ii]
    SCEN=SCEN.replace('+', '_')
    filetitle = SCEN+'.csv'
    filetitle = os.path.join(testfolder, 'USA', filetitle)
    A = df.unstack(level=1).iloc[ii]

    A.name = 'new_Installed_Capacity_[MW]'
    A = pd.DataFrame(A)
    A.index=pd.PeriodIndex(A.index, freq='A')
    A = A.resample('Y').asfreq()
    A = A['new_Installed_Capacity_[MW]'].fillna(0).groupby(A['new_Installed_Capacity_[MW]'].notna().cumsum()).transform('mean')    
    A = pd.DataFrame(A)
    A['new_Installed_Capacity_[MW]'] = A['new_Installed_Capacity_[MW]'] * 1000   # ReEDS file is in GW.
    # Add other columns
    A = pd.concat([A, baseline.reindex(A.index)], axis=1)
    A.loc['2050']['new_Installed_Capacity_[MW]'] = A.loc['2050']['new_Installed_Capacity_[MW]']/2   # Dividing last value by 2
    new_row = A[-1:]
    new_row.index = new_row.index.shift(periods=1)   #Shifting so new row is 2051
    A = pd.concat([A,pd.concat([new_row])])              
    A.index = A.index.shift(periods=-1)          # Shifting back so it goes from 2009-2050

    
    header = "year,new_Installed_Capacity_[MW],mod_eff,mod_reliability_t50,mod_reliability_t90,"\
    "mod_degradation,mod_lifetime,mod_MFG_eff,mod_EOL_collection_eff,mod_EOL_collected_recycled,"\
    "mod_Repowering,mod_Repairing\n"\
    "year,MW,%,years,years,%,years,%,%,%,%,%\n"

    with open(filetitle, 'w', newline='') as ict:
    # Write the header lines, including the index variable for
    # the last one if you're letting Pandas produce that for you.
    # (see above).
        for line in header:
            ict.write(line)

        #    savedata.to_csv(ict, index=False)
        A.to_csv(ict, header=False)